In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Dropout
from sklearn.metrics import mean_squared_error

In [2]:
# Load the dataset
data = pd.read_csv(r"C:\Users\HP\OneDrive\Desktop\DL\final_exam practical dataset.csv")
data['date'] = pd.to_datetime(data['date'], format='%d-%m-%Y')
data.set_index('date', inplace=True)

In [3]:
# Select the 'meantemp' column for forecasting
temperature_data = data['meantemp'].values.reshape(-1, 1)

In [4]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(temperature_data)


In [5]:
# Prepare the data for RNN
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

In [6]:
# Define the time step
time_step = 10
X, y = create_dataset(scaled_data, time_step)

# Reshape X to be [samples, time steps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Split the dataset into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build the RNN model
model = Sequential()
model.add(SimpleRNN(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(SimpleRNN(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))

C:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Inverse transform to get actual values
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

# Calculate RMSE
train_rmse = np.sqrt(mean_squared_error(y_train, train_predict))
test_rmse = np.sqrt(mean_squared_error(y_test, test_predict))
print(f'Train RMSE: {train_rmse}, Test RMSE: {test_rmse}')


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - loss: 0.2250
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1956 
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1850 
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1156 
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0873 
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0741 
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0492  
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0407 
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0655 
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0434 
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0508 
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0631 
Epoch 13/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0295 
Epoch 14/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0482 
Epoch 15/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0461 
Epoch 16/100
3/3 ━

In [8]:
# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(data.index, temperature_data, label='Actual Temperature', color='blue')
train_size = len(train_predict)
test_size = len(test_predict)
plt.plot(data.index[:train_size], train_predict, label='Train Predictions', color='orange')
plt.plot(data.index[train_size:train_size + test_size], test_predict, label='Test Predictions', color='red')
plt.title('Temperature Forecasting with RNN')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()

In [9]:
# Forecasting future temperatures
future_steps = 30  # Number of days to forecast
last_data = scaled_data[-time_step:].reshape(1, time_step, 1)

future_predictions = []

for _ in range(future_steps):
    next_pred = model.predict(last_data)
    future_predictions.append(next_pred[0, 0])
    last_data = np.append(last_data[:, 1:, :], next_pred.reshape(1, 1, 1), axis=1)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━

In [10]:
# Inverse transform future predictions
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Create dates for future predictions
last_date = data.index[-1]
future_dates = [last_date + pd.Timedelta(days=i) for i in range(1, future_steps + 1)]

# Create a DataFrame for future predictions
future_df = pd.DataFrame(data=future_predictions, index=future_dates, columns=['Predicted Temperature'])

# Save to CSV
future_df.to_csv(r'C:\Users\HP\OneDrive\Desktop\DL\future_temperature_predictions_rnn.csv')
# Display future predictions
print("Future Temperature Predictions:")
print(future_df)


Future Temperature Predictions:
            Predicted Temperature
2017-04-25              32.118359
2017-04-26              32.113636
2017-04-27              32.208897
2017-04-28              32.326752
2017-04-29              32.393990
2017-04-30              32.356094
2017-05-01              32.395718
2017-05-02              32.301098
2017-05-03              32.293312
2017-05-04              32.231518
2017-05-05              32.211155
2017-05-06              32.184841
2017-05-07              32.176815
2017-05-08              32.161221
2017-05-09              32.160213
2017-05-10              32.144638
2017-05-11              32.140202
2017-05-12              32.125473
2017-05-13              32.116943
2017-05-14              32.104340
2017-05-15              32.095547
2017-05-16              32.085152
2017-05-17              32.077904
2017-05-18              32.069378
2017-05-19              32.063255
2017-05-20              32.056122
2017-05-21              32.050488
2017-05-22      

Comparison of LSTM and RNN Results for Weather Forecasting

1. Root Mean Square Error (RMSE)
•	LSTM Model 
o	Train RMSE: 11.27
o	Test RMSE: 11.63
•	RNN Model 
o	Train RMSE: 19.97
o	Test RMSE: 29.78

•	The LSTM model has significantly lower RMSE values for both training and testing datasets compared to the RNN model. This indicates that the LSTM model provides better accuracy in forecasting temperature, as lower RMSE values signify closer predictions to the actual values.

2. Future Temperature Predictions

LSTM Model Predictions (April 25 - May 24, 2017):
Predicted temperatures range from 11.00 to 12.69.
RNN Model Predictions (April 25 - May 24, 2017):
Predicted temperatures range from 32.04 to 32.12. 

   LSTM Predictions: 
•	The predicted temperatures for the upcoming days are relatively low, ranging from approximately 11.00 to 12.69 degrees.
   RNN Predictions: 
•	In contrast, the RNN model predicts much higher temperatures, starting from around 32.11 degrees and remaining consistently above 32 degrees for the forecasted period.

•	The LSTM model predicts temperatures around 11°C, while the RNN model predicts temperatures around 32°C. This large discrepancy suggests that the RNN model may not be capturing the underlying patterns in the data effectively, potentially leading to overfitting or poor generalization.

Summary of Findings
•	Accuracy: The LSTM model outperforms the RNN model in both training and testing phases, as evidenced by lower RMSE values.
•	Prediction Range: The temperature predictions from the LSTM model are significantly lower than those from the RNN model, which may indicate that the RNN is not capturing the underlying patterns in the data effectively. 
